In [1]:
# -*- coding: utf-8 -*-
"""MRC_LSTM.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1D0cveLabP1nYyB1wGipzScvtj7ybbd0S
"""

"""## Imports and multi-worker config"""


from preprocessing import process
from model import create_model
from contextlib import redirect_stdout
from plotting import save_loss_accuracy_plots
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow as tf
import io
import time
import numpy as np
import pandas as pd
import json
import os
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'


"""## Constants"""

model_params_file = "./parameters/params.json"
timestamp = time.strftime("%Y%m%d-%H%M%S")

# Load parameters from the JSON file
with open(model_params_file, 'r') as file:
    params = json.load(file)

# Access the parameters
filename = params['filename']
window_size = params['window_size']
num_features = params['num_features']
batch_size = params['batch_size']
epochs = params['epochs']
initial_learning_rate = params['initial_learning_rate']
filter_size = params['filter_size']
external_filter_size = params['external_filter_size']
l2_reg = params['l2_reg']
bidirectional = params['bidirectional']
interval = params['interval']
train_size = params['train_size']
validation_size = params['validation_size']
test_size = params['test_size']
nr_of_labels = params['nr_of_labels']

assert (train_size + test_size + validation_size) == 1

base_results_path = './results'

"""## Loading data from CSV"""

data = pd.read_csv(filename)

"""## Preprocessing"""

data = process(data, interval, False)

"""## Extracting features and labels"""

data = data[['Coppock', 'RSI', 'StochRSI', 'ROC', 'MACD_Line', 'Signal_Line', 'MACD', 'null', '-null', 'long', 'short']]

"""## Reshaping to fit convolutional layers"""

num_samples = len(data) - window_size + 1
dataset = tf.data.Dataset.from_tensor_slices(data)
dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
# window[from 1 to windowsize -1, all columns except for the last two] window[only the last one, last two columns]
dataset = dataset.map(lambda window: (window[:-1, :-nr_of_labels], tf.squeeze(window[-1:, -nr_of_labels:])))

"""## Split the data"""

# One time data shuffle
shuffle_buffer_size = num_samples
# dataset = dataset.shuffle(shuffle_buffer_size, reshuffle_each_iteration=False) # hope fixed

# Splitting into training and validation sets
train_size = int(train_size * num_samples)
validation_size = int(validation_size * num_samples)
test_size = int(test_size * num_samples)

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(validation_size)
test_dataset = dataset.skip(train_size).skip(validation_size)


2023-07-07 11:25:40.064047: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-07 11:25:40.090230: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-07 11:25:40.466833: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-07 11:25:42.771619: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

In [ ]:
for X, y in dataset:
 print("Feature:", X.numpy(), "Target:", y.numpy())

In [ ]:

# Train Data shuffle on each iteration
shuffle_buffer_size = int(num_samples * train_size)
# train_dataset = train_dataset.shuffle(shuffle_buffer_size, reshuffle_each_iteration=False) # hope fixed

# Grouping windows into batches
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

"""## Model object and strategy"""

model = create_model(window_size, num_features, bidirectional, filter_size, l2_reg, external_filter_size, nr_of_labels)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=initial_learning_rate
)

if nr_of_labels == 2:
    loss = tf.keras.losses.BinaryCrossentropy()
else:
    loss = tf.keras.losses.CategoricalCrossentropy()

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'], run_eagerly=True)

"""## Logging, saving and plotting"""

run_path = os.path.join(base_results_path, timestamp)
os.makedirs(run_path)

# Model summary
summary_file = os.path.join(run_path, 'model_summary.txt')

buffer = io.StringIO()
with redirect_stdout(buffer):
    model.summary()
summary_string = buffer.getvalue()

with open(summary_file, 'w') as file:
    model_summary = model.summary()

    file.write(summary_string)

# Checkpoints
checkpoint_path = os.path.join(run_path, 'checkpoints')
os.makedirs(checkpoint_path)
checkpoint_file_path = os.path.join(checkpoint_path, 'checkpoint.h5')

# Logs
logs_path = os.path.join(run_path, "logs")
os.makedirs(logs_path)
logs_file_path = os.path.join(logs_path, "logs.log")

logging.basicConfig(filename=logs_file_path, level=logging.DEBUG)
logging.getLogger().addHandler(logging.FileHandler(logs_file_path))

# Training progress
progress_path = os.path.join(run_path, "progress")
os.makedirs(progress_path)
progress_file_path = os.path.join(progress_path, "progress.log")

# Plotting
plot_path = os.path.join(run_path, "plots")
os.makedirs(plot_path)

# Parameters
params_path = os.path.join(run_path, "params")
os.makedirs(params_path)
params_path_file = os.path.join(params_path, "params.json")
command = f'cp {model_params_file} {params_path_file}'
os.popen(command)

"""## Callbacks"""

# Define the learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[250, 500, 1000, 1500],
    values=[initial_learning_rate, initial_learning_rate * 0.3, initial_learning_rate * 0.3 * 0.3,
            initial_learning_rate * 0.3 * 0.3 * 0.3, initial_learning_rate * 0.3 * 0.3 * 0.3 * 0.3]
)

# Set the early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)  # zmienic na 5

# Define a custom callback to clear GPU memory


class ClearMemoryCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        tf.keras.backend.clear_session()


# Create an instance of the custom callback
clear_memory_callback = ClearMemoryCallback()

# Set checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_file_path,
    save_weights_only=True,
    save_freq='epoch',
    period=5  # Save every 5 epochs
)

# Create the CSVLogger callback to save training logs
csv_logger_callback = CSVLogger(progress_file_path)

"""## Learning process"""

# Devices
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

# Train the model with learning rate schedule and early stopping
history = model.fit(train_dataset, validation_data=val_dataset, epochs=epochs, callbacks=[  # tf.keras.callbacks.LearningRateScheduler(lr_schedule),
    clear_memory_callback, early_stopping, checkpoint_callback, csv_logger_callback])

"""## Plotting anf saving"""

############################ Test ###############################
test_loss, test_accuracy = model.evaluate(test_dataset)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Saving
model_path = os.path.join(run_path, 'model')
os.makedirs(model_path)
model_file_path = os.path.join(model_path, 'model.h5')
model.save(model_file_path)

# Plotting
save_loss_accuracy_plots(history, plot_path)